# Digitale Nachlässe

Zunächst müssen die notwendigen Bibliotheken geladen und importiert werden.

* `os`: Diese Bibliothek wird verwendet, um mit dem Betriebssystem zu interagieren, z.B. um Dateien und Verzeichnisse zu durchsuchen.
* `pandas`: Eine leistungsstarke Bibliothek zur Datenanalyse und -manipulation.
* `datetime`: Wird verwendet, um Datums- und Zeitinformationen zu verarbeiten.
* `sys`: Ermöglicht den Zugriff auf System-spezifische Parameter und Funktionen.
* `requests`: Eine Bibliothek zum Senden von HTTP-Anfragen.
* `BeautifulSoup`: Ein Werkzeug zum Parsen von HTML- und XML-Dokumenten.

Um die Bibliotheken zunächst zu installieren, kann das Kokmmentar in der nächsten Zeile entfernt werden. Dann einfach die Zelle ausführen. 

In [1]:
# !pip install pandas requests beautifulsoup4 lxml openpyxl ollama

In [2]:
import os
import pandas as pd
from datetime import datetime
import requests
from bs4 import BeautifulSoup

## Funktion zum Sammeln der Informationen

Die folgende Funktion durchsucht ein angegebenes Verzeichnis (`directory_path`) und sammelt Informationen über jede Datei darin. Sie speichert diese Informationen in einer Liste von Wörterbüchern namens `file_info_list` und gibt dieses als Rückgabewert zurück. Für jede Datei legt sie ein eigenes Wörterbuch `file_info` an.

In [3]:
def get_file_info(directory_path):
    file_info_list = []
    for root, dirs, files in os.walk(directory_path):
        for i, file in enumerate(files, start=1):
            file_path = os.path.join(root, file)
            file_stats = os.stat(file_path)
            file_extension = os.path.splitext(file)[1].lower()
            file_type = get_file_type(file_path)
            file_info = {
                "Laufende Nummer": i,
                "Dateiname": file,
                "Dateiendung": file_extension,
                "Dateipfad": file_path,
                "Änderungsdatum": datetime.fromtimestamp(file_stats.st_mtime).strftime('%Y-%m-%d %H:%M:%S'),
                "Erstellungsdatum": datetime.fromtimestamp(file_stats.st_ctime).strftime('%Y-%m-%d %H:%M:%S'),
                "Dateigröße (Bytes)": file_stats.st_size,
                "Dateityp": file_type,
                "Bemerkungen": ""
            }
            file_info_list.append(file_info)
    return file_info_list

Falls ein [FITS-Server](https://projects.iq.harvard.edu/fits/fits-web-service) zur Verfügung steht, können die Kommentare in den beiden auskommentierten Zeilen wieder entfernt werden.

## Funktion zum Abrufen des Dateityps

Die nächste Funktion sendet eine Datei an einen FITS-Server, der den Dateityp bestimmt, und gibt diesen zurück. Wenn der Dienst den Dateityp nicht bestimmen kann, wird "Unbekannt" zurückgegeben. Die Funktion wird nicht benötigt, wenn kein FITS-Server zur Verfügung steht.

In [4]:
def get_file_type(file_path):
    url = "http://etc.dnb.de.appr/fits/examine" # Nur im Netz der DNB erreichbar
    with open(file_path, 'rb') as file:
        response = requests.post(url, files={'datafile': file}, verify=False)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'lxml-xml')
        format_element = soup.find('identity', format=True) # Durchsucht das XML nach der Angabe des Datetyps
        if format_element:
            return format_element['format']
    return "Unbekannt"

## Angaben sammeln und speichern

In welchem Verzeichnis sollen die Metadaten der Dateien bestimmt werden? Kommentar entfernen und den Pfad angeben.

In [5]:
# directory_path = "path/to/your/directory"
directory_path = "/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Altenhein_Arbeitskopie"

Als nächstes wird die Liste mit den Angaben zu den Dateien erstellt, in dem die oben definierte Funktion aufgerufen wird:

In [6]:
file_info_list = get_file_info(directory_path)

Diese Liste mit Wörterbüchern ist die Grundlage für einen Dataframe:

In [7]:
df = pd.DataFrame(file_info_list)
df

,Laufende Nummer,Dateiname,Dateiendung,Dateipfad,Änderungsdatum,Erstellungsdatum,Dateigröße (Bytes),Dateityp,Bemerkungen
0,1,Dies academicus 3.docx,.docx,/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Alten...,2021-07-23 08:24:28,2024-06-17 09:51:52,13679,Office Open XML Document,
1,2,Ernst Jandl.docx,.docx,/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Alten...,2022-06-08 16:41:30,2024-06-17 09:51:52,14128,Office Open XML Document,
2,3,Ernst Umlauff.docx,.docx,/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Alten...,2022-11-23 14:18:50,2024-06-17 09:51:52,26287,Office Open XML Document,
3,4,Hörbücher als Datei(1)(1).docx,.docx,/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Alten...,2022-11-21 13:12:00,2024-06-17 09:51:52,18903,Office Open XML Document,
4,5,Lebenslauf.docx,.docx,/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Alten...,2022-11-21 13:24:20,2024-06-17 09:51:53,13984,Office Open XML Document,
...,...,...,...,...,...,...,...,...,...
70,1,Janus-Bücher.docx,.docx,/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Alten...,2021-08-18 16:57:42,2024-06-17 09:52:05,14760,Office Open XML Document,
71,2,Oldenbourg.docx,.docx,/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Alten...,2022-11-22 15:56:42,2024-06-17 09:52:05,35470,Office Open XML Document,
72,3,Zu Horst Kliemann.docx,.docx,/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Alten...,2021-08-25 08:37:48,2024-06-17 09:52:05,17839,Office Open XML Document,
73,1,IndexerVolumeGuid,,/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Alten...,2021-12-07 14:18:44,2024-06-17 09:52:00,76,Plain text,


Zum Speichern des Dataframes kann eine Excel-Datei erstellt werden:

In [8]:
# output_file = "/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Altenhein.xlsx"
# df.to_excel(output_file, index=False)

## Text-Dateien extrahieren

Um anschließend alle Textdateien aus dem Dataframe zu extrahieren, kann ein einfacher, leicht anzupassnder Filter genutzt werden:

In [9]:
def filter_text_files(df):
    # Definiere die gewünschten Dateiendungen
    # text_extensions = ['.doc', '.docx', '.pdf', '.md', '.txt']
    text_extensions = ['.docx']
    
    # Filtere den Dataframe nach den gewünschten Dateiendungen
    df_text = df[df['Dateiendung'].isin(text_extensions)]
    
    return df_text

Jetzt kann der Filter angewandt werden:

In [10]:
df_text = filter_text_files(df)
df_text

,Laufende Nummer,Dateiname,Dateiendung,Dateipfad,Änderungsdatum,Erstellungsdatum,Dateigröße (Bytes),Dateityp,Bemerkungen
0,1,Dies academicus 3.docx,.docx,/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Alten...,2021-07-23 08:24:28,2024-06-17 09:51:52,13679,Office Open XML Document,
1,2,Ernst Jandl.docx,.docx,/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Alten...,2022-06-08 16:41:30,2024-06-17 09:51:52,14128,Office Open XML Document,
2,3,Ernst Umlauff.docx,.docx,/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Alten...,2022-11-23 14:18:50,2024-06-17 09:51:52,26287,Office Open XML Document,
3,4,Hörbücher als Datei(1)(1).docx,.docx,/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Alten...,2022-11-21 13:12:00,2024-06-17 09:51:52,18903,Office Open XML Document,
4,5,Lebenslauf.docx,.docx,/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Alten...,2022-11-21 13:24:20,2024-06-17 09:51:53,13984,Office Open XML Document,
...,...,...,...,...,...,...,...,...,...
68,11,Urs Jaeggi.docx,.docx,/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Alten...,2022-09-06 14:00:10,2024-06-17 09:52:04,19133,Office Open XML Document,
69,12,Zur Programmgeschichte von Luchterhand.docx,.docx,/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Alten...,2022-11-15 16:40:56,2024-06-17 09:52:04,13802,Office Open XML Document,
70,1,Janus-Bücher.docx,.docx,/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Alten...,2021-08-18 16:57:42,2024-06-17 09:52:05,14760,Office Open XML Document,
71,2,Oldenbourg.docx,.docx,/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Alten...,2022-11-22 15:56:42,2024-06-17 09:52:05,35470,Office Open XML Document,


Optional: Speichere den neuen Dataframe in einer Excel-Datei.

In [11]:
# df_text.to_excel('text_files.csv', index=False)

## Inhalt zusammenfassen

Damit nicht alle Dateien, die Texte enthalten, autoptisch durchgesehen werden müssen, lassen wir die Aufgabe Ollama mit Llama3 übernehmen. Dafür importieren wir zunächst das Hilfsskript `summarize.py`.

In [12]:
# from summarize import summarize_text
import ollama
import docx2txt

Anschließend erstellen wir eine Funktion, die über den Dataframe `df_text` interiert und die Dateien an Ollama weiterreicht. Ollama kann dafür lokal oder extern auf einem Server laufen.

In [13]:
def summarize_dataframe(df_text):
    # Leere Listen, um die Ergebnisse zu speichern
    file_paths = []
    summaries = []

    # Iteriere über die Zeilen des DataFrames
    for index, row in df_text.iterrows():
        file_path = row['Dateipfad']
        text = docx2txt.process(file_path)
        prompt = "Deine Aufgabe ist es, den gegebenen Text in ungefähr 300 Wörtern zusammenzufassen. Gebe nur die Zusammenfassung wieder ohne weitere Angaben."
        response = ollama.chat(
        model="llama3",
        messages=[
            {
                'role': 'user',
                'content': f"{prompt}. Das ist der Text: {text}"
            },
        ],
        )
        summary = response['message']['content']
        # summary = summarize_text(text)  # Aufruf der bereits existierenden Funktion

        # Speichere die Ergebnisse
        file_paths.append(file_path)
        summaries.append(summary)

    # Erstelle einen neuen DataFrame mit den Ergebnissen
    df_summarized = pd.DataFrame({
        'Dateipfad': file_paths,
        'Zusammenfassung': summaries
    })

    return df_summarized

In [16]:
#df_test = df_text.head(10)
df_summarized = summarize_dataframe(df_text)
df_summarized

,Dateipfad,Zusammenfassung
0,/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Alten...,Die Schwere Heimatflakbatterie in Leipzig-Lind...
1,/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Alten...,Hier ist die Zusammenfassung:\n\nErnst Jandl (...
2,/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Alten...,What a fascinating topic! The history of the G...
3,/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Alten...,Das Verschwinden von Tonträgern: Die Entwicklu...
4,/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Alten...,Hier ist die Zusammenfassung:\n\nHans Altenhei...
...,...,...
61,/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Alten...,Der Schweizer Soziologie-Professor Urs Jaeggi ...
62,/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Alten...,Der literarische Programmverlag Luchterhand wu...
63,/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Alten...,Hier ist die Zusammenfassung:\n\nDie Aufgabe l...
64,/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Alten...,It appears that you have provided a list of pe...


In [17]:
df_merged = pd.merge(df_text, df_summarized, on='Dateipfad', how='inner')
df_merged

,Laufende Nummer,Dateiname,Dateiendung,Dateipfad,Änderungsdatum,Erstellungsdatum,Dateigröße (Bytes),Dateityp,Bemerkungen,Zusammenfassung
0,1,Dies academicus 3.docx,.docx,/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Alten...,2021-07-23 08:24:28,2024-06-17 09:51:52,13679,Office Open XML Document,,Die Schwere Heimatflakbatterie in Leipzig-Lind...
1,2,Ernst Jandl.docx,.docx,/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Alten...,2022-06-08 16:41:30,2024-06-17 09:51:52,14128,Office Open XML Document,,Hier ist die Zusammenfassung:\n\nErnst Jandl (...
2,3,Ernst Umlauff.docx,.docx,/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Alten...,2022-11-23 14:18:50,2024-06-17 09:51:52,26287,Office Open XML Document,,What a fascinating topic! The history of the G...
3,4,Hörbücher als Datei(1)(1).docx,.docx,/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Alten...,2022-11-21 13:12:00,2024-06-17 09:51:52,18903,Office Open XML Document,,Das Verschwinden von Tonträgern: Die Entwicklu...
4,5,Lebenslauf.docx,.docx,/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Alten...,2022-11-21 13:24:20,2024-06-17 09:51:53,13984,Office Open XML Document,,Hier ist die Zusammenfassung:\n\nHans Altenhei...
...,...,...,...,...,...,...,...,...,...,...
61,11,Urs Jaeggi.docx,.docx,/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Alten...,2022-09-06 14:00:10,2024-06-17 09:52:04,19133,Office Open XML Document,,Der Schweizer Soziologie-Professor Urs Jaeggi ...
62,12,Zur Programmgeschichte von Luchterhand.docx,.docx,/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Alten...,2022-11-15 16:40:56,2024-06-17 09:52:04,13802,Office Open XML Document,,Der literarische Programmverlag Luchterhand wu...
63,1,Janus-Bücher.docx,.docx,/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Alten...,2021-08-18 16:57:42,2024-06-17 09:52:05,14760,Office Open XML Document,,Hier ist die Zusammenfassung:\n\nDie Aufgabe l...
64,2,Oldenbourg.docx,.docx,/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Alten...,2022-11-22 15:56:42,2024-06-17 09:52:05,35470,Office Open XML Document,,It appears that you have provided a list of pe...


In [18]:
df_merged.to_excel('/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Altenheit_summarized.xlsx')

In [19]:
df_complete = pd.merge(df, df_summarized, on='Dateipfad', how='outer')
df_complete

,Laufende Nummer,Dateiname,Dateiendung,Dateipfad,Änderungsdatum,Erstellungsdatum,Dateigröße (Bytes),Dateityp,Bemerkungen,Zusammenfassung
0,1,Antiquariat.docx,.docx,/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Alten...,2020-11-30 14:01:44,2024-06-17 09:51:55,13889,Office Open XML Document,,Hier ist die Zusammenfassung des Textes:\n\nDe...
1,2,HK.docx,.docx,/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Alten...,2022-11-15 16:35:46,2024-06-17 09:51:55,20234,Office Open XML Document,,Hier ist die Zusammenfassung:\n\nDie Historisc...
2,3,Selbstzerstörung 2.docx,.docx,/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Alten...,2022-09-07 09:43:42,2024-06-17 09:51:55,20211,Office Open XML Document,,Hier ist die Zusammenfassung des Textes:\n\nIn...
3,5,Vorschlag zur Fertigstellung der.docx,.docx,/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Alten...,2019-09-02 14:06:00,2024-06-17 09:51:56,18808,Office Open XML Document,,"Die Fertigstellung der ""Geschichte des deutsch..."
4,6,Zentrum für BwState of the Art.docx,.docx,/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Alten...,2018-10-23 10:39:16,2024-06-17 09:51:56,14558,Office Open XML Document,,Die Historische Kommission des Börsenvereins v...
...,...,...,...,...,...,...,...,...,...,...
70,3,desktop.ini,.ini,/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Alten...,2022-03-21 13:13:42,2024-06-17 09:52:04,168,Plain text,,NaN
71,6,krise.docx,.docx,/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Alten...,2022-11-22 09:44:34,2024-06-17 09:52:05,13829,Office Open XML Document,,Die Luchterhand-Autoren begegnen sich im Frank...
72,1,Janus-Bücher.docx,.docx,/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Alten...,2021-08-18 16:57:42,2024-06-17 09:52:05,14760,Office Open XML Document,,Hier ist die Zusammenfassung:\n\nDie Aufgabe l...
73,2,Oldenbourg.docx,.docx,/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Alten...,2022-11-22 15:56:42,2024-06-17 09:52:05,35470,Office Open XML Document,,It appears that you have provided a list of pe...


In [20]:
df_complete.to_excel('/mnt/b/Digitale_Nachlaesse_und_Vorlaesse/Altenheit_komplett.xlsx')